In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cols=['sentiment','id','date','query_string','user','text']
data=pd.read_csv('/content/drive/MyDrive/NOTES BACKUP/DATAsets/training_1600000_processed_noemoticon.csv',encoding='latin1', names=cols)

In [ ]:
data.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   sentiment     1600000 non-null  int64 
 1   id            1600000 non-null  int64 
 2   date          1600000 non-null  object
 3   query_string  1600000 non-null  object
 4   user          1600000 non-null  object
 5   text          1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
data.columns

Index(['sentiment', 'id', 'date', 'query_string', 'user', 'text'], dtype='object')

In [ ]:
data.isnull().sum()

sentiment       0
id              0
date            0
query_string    0
user            0
text            0
dtype: int64

In [ ]:
data.describe()

,sentiment,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [ ]:
def format_text(df,col):
  #Remove @ tags
  comp_df = df.copy()
    
  # remove all the punctuation
  comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

  #Remove URL
  comp_df[col] = comp_df[col].str.replace(r"http\S+", "")

  #Remove # tag and the following words
  comp_df[col] = comp_df[col].str.replace(r'#\w+',"")

  #Remove all non-character
  comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

  # Remove extra space
  comp_df[col] = comp_df[col].str.replace(r'( +)'," ")
  comp_df[col] = comp_df[col].str.strip()

  # Change to lowercase
  comp_df[col] = comp_df[col].str.lower()

  return comp_df

In [ ]:
data=format_text(data,'text')


In [ ]:
X=data['text']
y=data['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X_train

22083      just bought new tax disc for my car the first ...
244558     as i dont like it its too bright for me to go ...
199331     awww we all just met what project are u workin...
1508960    lesson of the night dont watch shows about mur...
1529757    i still have a lot of nannying work several ti...
                                 ...                        
152315        loving the zune hating the lack of canada love
963395     sounds like it was a blast have a safe flight ...
117952     i feel bad for the zines i still need to send ...
1484405    just finished dust and moonlight this week so ...
305711     but i want it to be real you would level up yo...
Name: text, Length: 1200000, dtype: object

In [ ]:
c_vec=CountVectorizer(max_features=20000)
vec=c_vec.fit_transform()

In [ ]:
stop_w=text.ENGLISH_STOP_WORDS.union()
tfid_vec=TfidfVectorizer(stop_words=stop_w, binary=False, token_pattern=r"(?u)\b\w\w+\b")

In [ ]:
tfid_vec.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, sublinear_tf=False,
                token_pa

In [ ]:
X_train=tfid_vec.transform(X_train)
X_test=tfid_vec.transform(X_test)

In [ ]:
len(tfid_vec.get_feature_names())

339928

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
clf=LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
ac=accuracy_score(y_test, y_pred)*100
print(ac, "%")

77.57325 %


In [ ]:
import pickle
pickle.dump(clf, open("/content/clf.pkl", "wb"))
pickle.dump(c_vec,open("/content/vectorize2.pkl", "wb"))